In [1]:
import pandas as pd
import numpy as np
import itertools
import pandas_gbq
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
*
FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
# WHERE A.Year=2024
# AND A.GP="Monaco Grand Prix"
# AND A.DRIVER='HAM'
ORDER BY LapNumber, LapStartTime
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [4]:
track2=track3[(track3["GP"]=='Dutch Grand Prix')&(track3["Year"]==2024)]

In [5]:
track2.head()
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [6]:
track2['LapTime']= pd.to_timedelta(track2["LapTime"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_26949/4173085553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track2['LapTime']= pd.to_timedelta(track2["LapTime"])


In [7]:
# track2['LapTime'].dt.total_seconds().min()*1.07
track2=track2[track2['LapNumber']!=1.0]
track2.loc[:, "LapTime (s)"] = track2["LapTime"].dt.total_seconds()

In [8]:
quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
quicklaps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s)
20590,0 days 01:00:11.819000,VER,1,0 days 00:01:16.106000,2.0,1.0,NaT,NaT,0 days 00:00:26.012000,0 days 00:00:27.179000,0 days 00:00:22.915000,0 days 00:59:21.725000,0 days 00:59:48.904000,0 days 01:00:11.819000,281.0,256.0,292.0,285.0,True,MEDIUM,2.0,True,Red Bull Racing,0 days 00:58:55.713000,NaT,1,1.0,False,,False,True,2024,Dutch Grand Prix,76.106
20592,0 days 01:00:12.810000,NOR,4,0 days 00:01:16.134000,2.0,1.0,NaT,NaT,0 days 00:00:26.061000,0 days 00:00:27.155000,0 days 00:00:22.918000,0 days 00:59:22.731000,0 days 00:59:49.886000,0 days 01:00:12.804000,283.0,262.0,299.0,282.0,True,MEDIUM,2.0,True,McLaren,0 days 00:58:56.676000,NaT,1,2.0,False,,False,True,2024,Dutch Grand Prix,76.134
20594,0 days 01:00:13.916000,RUS,63,0 days 00:01:16.389000,2.0,1.0,NaT,NaT,0 days 00:00:26.349000,0 days 00:00:27.159000,0 days 00:00:22.881000,0 days 00:59:23.864000,0 days 00:59:51.023000,0 days 01:00:13.904000,285.0,266.0,302.0,298.0,True,MEDIUM,2.0,True,Mercedes,0 days 00:58:57.527000,NaT,1,3.0,False,,False,True,2024,Dutch Grand Prix,76.389
20596,0 days 01:00:14.676000,PIA,81,0 days 00:01:16.240000,2.0,1.0,NaT,NaT,0 days 00:00:26.087000,0 days 00:00:27.427000,0 days 00:00:22.726000,0 days 00:59:24.515000,0 days 00:59:51.942000,0 days 01:00:14.668000,286.0,275.0,315.0,287.0,True,MEDIUM,2.0,True,McLaren,0 days 00:58:58.436000,NaT,1,4.0,False,,False,True,2024,Dutch Grand Prix,76.240
20597,0 days 01:00:15.815000,LEC,16,0 days 00:01:16.695000,2.0,1.0,NaT,NaT,0 days 00:00:26.633000,0 days 00:00:27.274000,0 days 00:00:22.788000,0 days 00:59:25.722000,0 days 00:59:52.996000,0 days 01:00:15.784000,284.0,274.0,304.0,300.0,True,MEDIUM,2.0,True,Ferrari,0 days 00:58:59.120000,NaT,1,5.0,False,,False,True,2024,Dutch Grand Prix,76.695


In [9]:
transformed_laps = quicklaps.copy()
transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

# order the team from the fastest (lowest median lap time) to slower
team_order = (
    transformed_laps[["Team", "LapTime (s)"]].groupby("Team").median()["LapTime (s)"].sort_values().index
)
print(team_order)

Index(['McLaren', 'Ferrari', 'Red Bull Racing', 'Mercedes', 'Aston Martin',
       'Alpine', 'RB', 'Williams', 'Haas F1 Team', 'Kick Sauber'],
      dtype='object', name='Team')


In [10]:
transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('median')
transformed_laps.tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s),median
581283,0 days 02:28:48.654000,PIA,81,0 days 00:01:15.092000,72.0,2.0,NaT,NaT,0 days 00:00:25.183000,0 days 00:00:27.421000,0 days 00:00:22.488000,0 days 02:27:58.737000,0 days 02:28:26.158000,0 days 02:28:48.646000,NaN,268.0,306.0,302.0,False,HARD,39.0,True,McLaren,0 days 02:27:33.562000,NaT,1,4.0,False,,False,True,2024,Dutch Grand Prix,75.092,75.0855
581284,0 days 02:28:53.454000,SAI,55,0 days 00:01:14.631000,72.0,2.0,NaT,NaT,0 days 00:00:25.328000,0 days 00:00:27.091000,0 days 00:00:22.212000,0 days 02:28:04.112000,0 days 02:28:31.203000,0 days 02:28:53.415000,289.0,266.0,306.0,307.0,False,HARD,42.0,True,Ferrari,0 days 02:27:38.823000,NaT,1,5.0,False,,False,True,2024,Dutch Grand Prix,74.631,75.2405
581285,0 days 02:29:00.859000,PER,11,0 days 00:01:14.956000,72.0,2.0,NaT,NaT,0 days 00:00:25.314000,0 days 00:00:27.143000,0 days 00:00:22.499000,0 days 02:28:11.196000,0 days 02:28:38.339000,0 days 02:29:00.838000,292.0,268.0,307.0,310.0,False,HARD,43.0,True,Red Bull Racing,0 days 02:27:45.903000,NaT,1,6.0,False,,False,True,2024,Dutch Grand Prix,74.956,75.2870
581286,0 days 02:29:05.934000,RUS,63,0 days 00:01:14.459000,72.0,3.0,NaT,NaT,0 days 00:00:25.158000,0 days 00:00:26.965000,0 days 00:00:22.336000,0 days 02:28:16.621000,0 days 02:28:43.586000,0 days 02:29:05.922000,NaN,270.0,309.0,310.0,False,SOFT,21.0,False,Mercedes,0 days 02:27:51.475000,NaT,1,7.0,False,,False,True,2024,Dutch Grand Prix,74.459,75.4350
581287,0 days 02:29:10.916000,HAM,44,0 days 00:01:15.688000,72.0,3.0,NaT,NaT,0 days 00:00:25.463000,0 days 00:00:27.166000,0 days 00:00:23.059000,0 days 02:28:20.684000,0 days 02:28:47.850000,0 days 02:29:10.909000,292.0,269.0,261.0,291.0,False,SOFT,24.0,True,Mercedes,0 days 02:27:55.228000,NaT,1,8.0,False,,False,True,2024,Dutch Grand Prix,75.688,75.4350


In [11]:
fig_pace=px.box(
    transformed_laps.sort_values(by=["median","LapNumber"]),
    x="Team",
    y="LapTime (s)",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace Plot for the {} {}</b>".format(year,gp),
    # palette=team_palette,
    # whiskerprops=dict(color="white"),
    # boxprops=dict(edgecolor="white"),
    # medianprops=dict(color="grey"),
    # capprops=dict(color="white"),
    height=700, width=1200,

color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)
fig_pace.update_traces(opacity=1)
fig_pace.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    margin=dict(l=60, r=5, t=50, b=60),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pace.show()

In [12]:
fig_pace.write_html("{} {} Team Pace Plot.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [13]:
#Get MINIMUM OF THE MEDIAN LAP TIMES

fastest_lap = transformed_laps["median"].min()
transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
transformed_laps_relative_median=transformed_laps[["Team","Delta","Delta_Percent"]].drop_duplicates()
transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
transformed_laps_relative_median

,Team,Delta,Delta_Percent
20592,McLaren,0.0000,100.0%
20597,Ferrari,0.1550,100.206%
20590,Red Bull Racing,0.2015,100.268%
20594,Mercedes,0.3495,100.465%
20601,Aston Martin,1.2500,101.665%
20600,Alpine,1.2970,101.727%
20605,RB,1.4155,101.885%
31396,Williams,1.4655,101.952%
20604,Haas F1 Team,1.5415,102.053%
20607,Kick Sauber,2.0625,102.747%


In [14]:
for index, row in transformed_laps_relative_median.iterrows():
  if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
    transformed_laps_relative_median.loc[index, 'Delta']=0.001
transformed_laps_relative_median

,Team,Delta,Delta_Percent
20592,McLaren,0.0010,100.0%
20597,Ferrari,0.1550,100.206%
20590,Red Bull Racing,0.2015,100.268%
20594,Mercedes,0.3495,100.465%
20601,Aston Martin,1.2500,101.665%
20600,Alpine,1.2970,101.727%
20605,RB,1.4155,101.885%
31396,Williams,1.4655,101.952%
20604,Haas F1 Team,1.5415,102.053%
20607,Kick Sauber,2.0625,102.747%


In [15]:
fig_median=px.bar(
    transformed_laps_relative_median,
    x="Team",
    y="Delta",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace (Median Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent",
    height=700, width=1200,
color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)

fig_median.update_traces(textposition='outside')

fig_median.update_traces(opacity=0.85)

fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

# fig['layout']['yaxis']['autorange'] = "reversed"

fig_median.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_median.update_xaxes(title_text='')      # Remove axis titles
fig_median.update_yaxes(title_text='')

fig_median.update_xaxes(
        title_standoff = 75
)

fig_median.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_median.show()

In [16]:
fig_median.write_html("{} {} Team Median Plot.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [17]:
#Get MINIMUM OF ALL LAP TIMES
transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('min')

fastest_lap_all = transformed_laps["min_all"].min()
transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
transformed_laps_relative_min_all=transformed_laps[["Team","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
transformed_laps_relative_min_all

,Team,Delta_Min_All,Delta_Percent_All
20592,McLaren,0.000,100.0%
20594,Mercedes,0.061,100.083%
20597,Ferrari,0.300,100.406%
31396,Williams,0.617,100.836%
20590,Red Bull Racing,0.770,101.043%
20601,Aston Martin,0.941,101.275%
20600,Alpine,1.038,101.406%
20604,Haas F1 Team,1.137,101.54%
20605,RB,1.405,101.903%
20607,Kick Sauber,1.907,102.583%


In [18]:
for index, row in transformed_laps_relative_min_all.iterrows():
  if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
    transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
transformed_laps_relative_min_all

,Team,Delta_Min_All,Delta_Percent_All
20592,McLaren,0.001,100.0%
20594,Mercedes,0.061,100.083%
20597,Ferrari,0.300,100.406%
31396,Williams,0.617,100.836%
20590,Red Bull Racing,0.770,101.043%
20601,Aston Martin,0.941,101.275%
20600,Alpine,1.038,101.406%
20604,Haas F1 Team,1.137,101.54%
20605,RB,1.405,101.903%
20607,Kick Sauber,1.907,102.583%


In [19]:
fig_min=px.bar(
    transformed_laps_relative_min_all,
    x="Team",
    y="Delta_Min_All",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent_All",
    height=700, width=1200,
color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)

fig_min.update_traces(textposition='outside')

fig_min.update_traces(opacity=0.85)

fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

# fig['layout']['yaxis']['autorange'] = "reversed"

fig_min.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_min.update_xaxes(title_text='')      # Remove axis titles
fig_min.update_yaxes(title_text='')

fig_min.update_xaxes(
        title_standoff = 75
)

fig_min.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_min.update_yaxes(ticksuffix = "  ")
fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_min.show()

In [20]:
fig_min.write_html("{} {} Team Fastest Lap Plot.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [21]:
#Multi GP Plot

In [32]:
#2024
race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
# race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2024
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    track2['LapTime']= pd.to_timedelta(track2["LapTime"])
    track2=track2[track2['LapNumber']!=1.0]
    quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
    transformed_laps = quicklaps.copy()
    transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

    # order the team from the fastest (lowest median lap time) tp slower
    team_order = (
        transformed_laps[["Team", "LapTime (s)"]].groupby("Team").median()["LapTime (s)"].sort_values().index
    )
    print(team_order)
    transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('median')
    transformed_laps.tail()
    fig_pace=px.box(
        transformed_laps.sort_values(by=["median","LapNumber"]),
        x="Team",
        y="LapTime (s)",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace Plot for the {} {}</b>".format(year_multi,i),
        height=700, width=1200,

    color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            }
    )
    fig_pace.update_traces(opacity=1)
    fig_pace.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow",
        margin=dict(l=50, r=5, t=35, b=50)
    )
    fig_pace.write_html("{} {} Team Pace Plot.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')
    #Get MINIMUM OF THE MEDIAN LAP TIMES

    fastest_lap = transformed_laps["median"].min()
    transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
    transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
    transformed_laps_relative_median=transformed_laps[["Team","Delta","Delta_Percent"]].drop_duplicates()
    transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
    transformed_laps_relative_median
    for index, row in transformed_laps_relative_median.iterrows():
        if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
         transformed_laps_relative_median.loc[index, 'Delta']=0.001
    transformed_laps_relative_median
    fig_median=px.bar(
        transformed_laps_relative_median,
        x="Team",
        y="Delta",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace (Median Lap Times) for the {} {}</b>".format(year_multi,i),
        text="Delta_Percent",
        height=700, 
        width=1200,
        color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            }
    )

    fig_median.update_traces(textposition='outside')

    fig_median.update_traces(opacity=0.85)

    fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

    fig_median.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_median.update_xaxes(title_text='')      # Remove axis titles
    fig_median.update_yaxes(title_text='')

    fig_median.update_xaxes(
            title_standoff = 75
    )

    fig_median.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_median.write_html("{} {} Team Median Plot.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')
  #Get MINIMUM OF ALL LAP TIMES
    transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('min')
    fastest_lap_all = transformed_laps["min_all"].min()
    transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
    transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
    transformed_laps_relative_min_all=transformed_laps[["Team","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
    transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
    transformed_laps_relative_min_all
    for index, row in transformed_laps_relative_min_all.iterrows():
        if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
            transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
    fig_min=px.bar(
        transformed_laps_relative_min_all,
        x="Team",
        y="Delta_Min_All",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
        text="Delta_Percent_All",
        height=700, width=1200,
    color_discrete_map={
                     "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                    }
    )

    fig_min.update_traces(textposition='outside')

    fig_min.update_traces(opacity=0.85)

    fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

    # fig['layout']['yaxis']['autorange'] = "reversed"

    fig_min.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_min.update_xaxes(title_text='')      # Remove axis titles
    fig_min.update_yaxes(title_text='')

    fig_min.update_xaxes(
            title_standoff = 75
    )

    fig_min.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_min.update_yaxes(ticksuffix = "  ")
    fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_min.write_html("{} {} Team Fastest Lap Plot.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['Red Bull Racing', 'Ferrari', 'McLaren', 'Mercedes', 'Haas F1 Team',
       'Aston Martin', 'Kick Sauber', 'Alpine', 'RB', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'Ferrari', 'McLaren', 'Mercedes', 'Aston Martin',
       'Haas F1 Team', 'Kick Sauber', 'Alpine', 'Williams', 'RB'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['Ferrari', 'McLaren', 'Red Bull Racing', 'Mercedes', 'Aston Martin',
       'Haas F1 Team', 'Alpine', 'RB', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'McLaren', 'Mercedes', 'Ferrari', 'Aston Martin',
       'RB', 'Williams', 'Kick Sauber', 'Haas F1 Team', 'Alpine'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

...

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

Index(['Red Bull Racing', 'Mercedes', 'Williams', 'Ferrari', 'McLaren', 'RB',
       'Alpine', 'Kick Sauber', 'Aston Martin'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'Mercedes', 'McLaren', 'Aston Martin',
       'Haas F1 Team', 'RB', 'Alpine', 'Kick Sauber', 'Ferrari', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'McLaren', 'Ferrari', 'Mercedes', 'Alpine',
       'Haas F1 Team', 'RB', 'Aston Martin', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['McLaren', 'Ferrari', 'Mercedes', 'Red Bull Racing', 'Haas F1 Team',
       'Alpine', 'RB', 'Aston Martin', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')
Index(['McLaren', 'Mercedes', 'Red Bull Racing', 'Ferrari', 'Aston Martin',
       'Haas F1 Team', 'Williams', 'RB', 'Kick Sauber', 'Alpine'],
      dtype='object', name='Team')
Index(['Mercedes', 'Red Bull Racing', 'McLaren', 'Ferrari', 'Aston Martin',
       'RB', 'Haas F1 Team', 'Williams', 'Kick Sauber', 'Alpine'],
      dtype='object', name='Team')
Index(['Mercedes', 'McLaren', 'Ferrari', 'Red Bull Racing', 'Williams', 'RB',
       'Alpine', 'Aston Martin', 'Kick Sauber', 'Haas F1 Team'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [253]:
# #Get AVERAGE OF ALL LAP TIMES
# transformed_laps['avg_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform(np.mean)

# fastest_lap_avg = transformed_laps["avg_all"].min()
# transformed_laps['Delta_Avg_All']= transformed_laps["avg_all"] - (fastest_lap_avg)
# transformed_laps['Delta_Percent_Avg']= (transformed_laps["avg_all"]/(fastest_lap_avg)-1)*100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].round(3)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].astype(str)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+"%"
# transformed_laps_relative_avg_all=transformed_laps[["Team","Delta_Avg_All","Delta_Percent_Avg"]].drop_duplicates()
# transformed_laps_relative_avg_all=transformed_laps_relative_avg_all.sort_values("Delta_Avg_All")
# transformed_laps_relative_avg_all

In [254]:
# for index, row in transformed_laps_relative_avg_all.iterrows():
#   if transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']==0.0:
#     transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']=0.001
# transformed_laps_relative_avg_all

In [255]:
# fig=px.bar(
#     transformed_laps_relative_avg_all,
#     x="Team",
#     y="Delta_Avg_All",
#     color='Team',
#     template="xgridoff",
#     title="<b>Team Pace (Average Lap Times) for the {} {}</b>".format(year,gp),
#     text="Delta_Percent_Avg",
#     height=700, width=1200,
# color_discrete_map={
#                  "Alpine": "#0093cc",
#                  "Aston Martin": "#229971",
#                  "Ferrari": "#E80020",
#                  "Haas F1 Team": "#B6BABD",
#                  "Kick Sauber": "#52e252",
#                  "McLaren": "#FF8000",
#                  "Mercedes": "#27F4D2",
#                  "RB": "#6692FF",
#                  "Red Bull Racing": "#3671C6",
#                  "Williams": "#64C4FF" ,
#                  "Alfa Romeo":"#C92D4B",
#                  "AlphaTauri":"#5E8FAA",
#                  "Racing Point":"#F596C8",
#                  "Renault":"#FFF500",
#                  "Toro Rosso":"#469bff",
#                  "Force India":"#F596C8",
#                  "Sauber":"#9B0000"
#                  }
# )

# fig.update_traces(textposition='outside')

# fig.update_traces(opacity=0.85)

# fig.update_traces( marker_line_color='white',marker_line_width=1.5)

# fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_yaxes(range=[2, 0])

# # fig['layout']['yaxis']['autorange'] = "reversed"

# fig.update_layout(
#     xaxis={'side': 'top'},
#     yaxis={'side': 'left'}
# )

# fig.update_xaxes(title_text='')      # Remove axis titles
# fig.update_yaxes(title_text='')

# fig.update_xaxes(
#         title_standoff = 75
# )

# fig.update_layout(
#     title_x=0.5,
#     margin=dict(l=60, r=5, t=110, b=30),
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=14,
#         color="Black"
#     ),
#     title_font_family="PT Sans Narrow"
# )
# fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

# fig.show()

In [256]:
# fig.write_html("{} {} Team Avg Lap Plot.html".format(year,gp))